In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from utils import *
import struct
from math import *
from model_define import MyModel,NMSE_loss

In [2]:
Y_1 = np.loadtxt('Y_1.csv', delimiter=',')
Y_2 = np.loadtxt('Y_2.csv', delimiter=',')

In [3]:
def LMMSE(H_e,y_e):
    a = H_e*H_e.H + 1/10*np.mat(np.eye(2))
    W = H_e.H*a.I
    return (W*y_e).reshape(2)
def ZF(H_e,y_e):
    return (H_e.I*y_e).reshape(2)
def deModulation(Q):
    Qr=np.real(Q)
    Qi=np.imag(Q)
    bits=np.zeros([Q.shape[0],2])
    bits[:,0]=Qr>0
    bits[:,1]=Qi>0
    return bits.reshape([-1])  # This is just for QAM modulation
def decoder(Y,Pilotnum,K):
    N = Y.shape[0]
    mu = 2
    P = Pilotnum * 2
    Pilot_file_name = 'Pilot_' + str(P)
    bits = np.loadtxt(Pilot_file_name, delimiter=',')
    pilotValue = Modulation(bits, mu)
    pilotValue0 = pilotValue[0:P:2]
    pilotValue1 = pilotValue[1:P:2]
    
    Y_mat = np.transpose(np.reshape(Y,[N,256,8]),[0,2,1])
    Y_pilot = Y_mat[:,[0,1,4,5],:].reshape(N,1024)
    
    Y_antena0 = Y_mat[:,0:4,:]
    Y_antena0_pilot = Y_antena0[:,0,:]+1j*Y_antena0[:,1,:]

    Y_antena1 = Y_mat[:,4:8,:]
    Y_antena1_pilot = Y_antena1[:,0,:]+1j*Y_antena1[:,1,:]
    
    Y_antena0_data = Y_antena0[:,2,:]+1j*Y_antena0[:,3,:]
    Y_antena1_data = Y_antena1[:,2,:]+1j*Y_antena1[:,3,:]

    y = np.zeros([N,K,2]).astype(complex)
    y[:,:,0] = Y_antena0_data
    y[:,:,1] = Y_antena1_data
    
    model_input = keras.Input(shape=(1024))
    model_output = MyModel(model_input)
    model = keras.Model(inputs=model_input, outputs=model_output)
    model.compile(optimizer='adam', loss=NMSE_loss)
    model.load_weights(".model_"+str(Pilotnum)+".best.h5")
    print(Y_pilot.shape)
    out_H = model.predict(Y_pilot, batch_size=128)/20
    H_est = out_H[:,:,:,0]+1j*out_H[:,:,:,1]
    H_est00_t = np.hstack((H_est[:,0,:],np.zeros([N,K-32])))
    H_est00 = np.fft.fft(H_est00_t)
    H_est01_t = np.hstack((H_est[:,1,:],np.zeros([N,K-32])))
    H_est01 = np.fft.fft(H_est01_t)
    H_est10_t = np.hstack((H_est[:,2,:],np.zeros([N,K-32])))
    H_est10 = np.fft.fft(H_est10_t)
    H_est11_t = np.hstack((H_est[:,3,:],np.zeros([N,K-32])))
    H_est11 = np.fft.fft(H_est11_t)

    H_MIMO = np.zeros([N,K,2,2]).astype(complex)
    H_MIMO[:,:,0,0] = H_est00
    H_MIMO[:,:,1,0] = H_est01
    H_MIMO[:,:,0,1] = H_est10
    H_MIMO[:,:,1,1] = H_est11
    
    x = np.zeros([N,K,2]).astype(complex)
    for i in range(N):
        for j in range(K):
            H_e = np.mat(H_MIMO[i,j,:,:])
            y_e = np.mat(y[i,j,:]).T
            x[i,j,:] = LMMSE(H_e,y_e)
            
    X_est = np.zeros([N,1024])
    for i in range(N):
        X_est[i,0:512] = deModulation(x[i,:,0])
        X_est[i,512:1024] = deModulation(x[i,:,1])
    return X_est

In [4]:
K = 256
X_pre_1 = decoder(Y_1,32,K)
X_pre_2 = decoder(Y_2,8,K)

(10000, 1024)
(10000, 1024)


In [5]:
X_pre_1 = np.array(np.floor(X_pre_1 + 0.5), dtype=np.bool)
X_pre_2 = np.array(np.floor(X_pre_2 + 0.5), dtype=np.bool)

In [6]:
X_pre_1.tofile('X_pre_1.bin')
X_pre_2.tofile('X_pre_2.bin')